# itk - tests-1 - OPEN IN JUPYTER NOTEBOOK (ipywidgets)

Created: 2021.11.12 / Updated: 2021.11.12

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

import os
import sys
import pathlib
import glob

import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt

In [3]:
# add path to our moudles
sys.path.append(os.getcwd().split('assets')[0])

import utils
import functions1 as f1

# GLOBAL VARIABLES

In [4]:
HOME_DIR = pathlib.Path(os.getcwd()).parent.parent.parent
DATA_DIR = HOME_DIR / 'data'
RESULTS_DIR = DATA_DIR / 'results'

SUB = 'CC3'

---

# GET LIST ALL CC3 IMAGES

In [5]:
files = list(RESULTS_DIR.glob(f"*{SUB}*.nii.gz"))
#files

In [6]:
# print only names
_ = [print(f'{k+1}) {f.name}') for k, f in enumerate(sorted(files))]

1) CC3_t2.nii.gz
2) CC3_t2_roi.nii.gz
3) CC3_t2_roi_cm.nii.gz
4) CC3_t2_roi_dilated2.nii.gz
5) CC3_t2_roi_dilated_ball3.nii.gz


## LOAD ALL IMAGES

In [7]:
# image
img = nib.load(RESULTS_DIR / f'{SUB}_t2.nii.gz').get_fdata()
# roi
roi = nib.load(RESULTS_DIR / f'{SUB}_t2_roi.nii.gz').get_fdata()
# dilated 2
dil2 = nib.load(RESULTS_DIR / f'{SUB}_t2_roi_dilated2.nii.gz').get_fdata()
# ball3
bal3 = nib.load(RESULTS_DIR / f'{SUB}_t2_roi_dilated_ball3.nii.gz').get_fdata()

### CREATE LIST OFLOADED IMAGES AND LIST OF THEIR NAMES

In [8]:
f1.im_info(img, 'img')

*** IMG ***,	max=1640.0, min=0.0, aver=443.37, shape=(320, 320, 28), dtype=float64


# ITK

[Quick start guide - Filter parameters](https://itkpythonpackage.readthedocs.io/en/master/Quick_start_guide.html#filter-parameters)

[Quick start guide](https://itkpythonpackage.readthedocs.io/en/stable/Quick_start_guide.html)

In [9]:
import itk

In [10]:
file = str(RESULTS_DIR / f'{SUB}_t2.nii.gz')
print(type(file))
print(os.path.exists(file))


image = itk.imread(file)
image.shape

<class 'str'>
True


(28, 320, 320)

In [11]:
print(image.GetSpacing())
print(image.GetOrigin())
print(image.GetImageDimension())
print(image.GetNumberOfComponentsPerPixel())
print(image.GetDirection())
print(image.shape)

itkVectorD3 ([0.625, 0.625, 3.3])
itkPointD3 ([-102.158, 131.657, -34.0861])
3
1
<itk.itkMatrixPython.itkMatrixD33; proxy of <Swig Object of type 'itkMatrixD33 *' at 0x7fb56c670750> >
(28, 320, 320)


---

# convertion numpy <-> itk

[Convert ITK Data Structures to NumPy Arrays](https://blog.kitware.com/convert-itk-data-structures-to-numpy-arrays/)

In [12]:
array_view = itk.GetArrayViewFromImage(image)
array = itk.GetArrayFromImage(image)

In [13]:
f1.im_info(array_view,  'view')
f1.im_info(array, 'array')

*** VIEW ***,	max=1640, min=0, aver=443.37, shape=(28, 320, 320), dtype=int16
*** ARRAY ***,	max=1640, min=0, aver=443.37, shape=(28, 320, 320), dtype=int16


In [14]:
image_view = itk.GetImageViewFromArray(array_view)
image2 = itk.GetImageFromArray(array)

In [15]:
image2.GetSpacing()

itkVectorD3 ([1, 1, 1])

---

[median image filter](https://blog.kitware.com/itk-python-image-pixel-types/)

In [16]:
median = itk.MedianImageFilter.New(image, Radius = 2)
median.Update()

In [17]:
#itk.imwrite(median, "output_filename.nii.gz")

---

In [18]:
stat = itk.StatisticsImageFilter.New(image)
stat.Update()

In [19]:
print(stat.GetMaximum())
print(stat.GetMinimum())
print(stat.GetMean())
print(stat.GetVariance())
print(stat.GetSigma())

1640
0
443.36615687779016
106684.85782641271
326.62648059582176


In [21]:
from itkwidgets import view

In [22]:
view(median)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itk.itkImagePython.itkImageSS3; pr…

---

# RESAMPLE IMAGE (ITK)
# https://itk.org/ITKExamples/src/Filtering/ImageGrid/ResampleAnImage/Documentation.html

In [ ]:
#!/usr/bin/env python

import sys
import itk

sub = 'CC1'

input_file_name = str(RESULTS_DIR / f'{sub}_t2.nii.gz')
output_file_name = str(RESULTS_DIR / f'{sub}_t2_resampled.nii.gz')
scale = float(2)

input_image = itk.imread(input_file_name)
input_size = itk.size(input_image)
input_spacing = itk.spacing(input_image)
input_origin = itk.origin(input_image)
Dimension = input_image.GetImageDimension()

# We will scale the objects in the image by the factor `scale`; that is they
# will be shrunk (scale < 1.0) or enlarged (scale > 1.0).  However, the number
# of pixels for each dimension of the output image will equal the corresponding
# number of pixels in the input image, with cropping or padding as necessary.
# Furthermore, the physical distance between adjacent pixels will be the same
# in the input and the output images.  In contrast, if you want to change the
# resolution of the image without changing the represented physical size of the
# objects in the image, omit the transform and instead supply:
#
# output_size = [int(input_size[d] * scale) for d in range(Dimension)]
# output_spacing = [input_spacing[d] / scale for d in range(Dimension)]
# output_origin = [input_origin[d] + 0.5 * (output_spacing[d] - input_spacing[d])
#                  for d in range(Dimension)]

output_size = input_size
output_spacing = input_spacing
output_origin = input_origin
scale_transform = itk.ScaleTransform[itk.D, Dimension].New()
scale_transform_parameters = scale_transform.GetParameters()
for i in range(len(scale_transform_parameters)):
    scale_transform_parameters[i] = scale
scale_transform_center = [float(int(s / 2)) for s in input_size]
scale_transform.SetParameters(scale_transform_parameters)
scale_transform.SetCenter(scale_transform_center)

interpolator = itk.LinearInterpolateImageFunction.New(input_image)

resampled = itk.resample_image_filter(
    input_image,
    transform=scale_transform,
    interpolator=interpolator,
    size=output_size,
    output_spacing=output_spacing,
    output_origin=output_origin,
)

itk.imwrite(resampled, output_file_name)
print('done')

---

# histogram

[Histogram Creation and Bin Access](https://itk.org/ITKExamples/src/Numerics/Statistics/HistogramCreationAndBinAccess/Documentation.html)

[HistogramMatchingImageFilter](https://simpleitk.org/doxygen/latest/html/classitk_1_1simple_1_1HistogramMatchingImageFilter.html)

----

In [ ]:
utils.print_date(5)

In [ ]:
utils.save_notebook_as_html(file_name='cnr-04-cc3', cleaning_delay=2)